In [1]:
import numpy as np
import pandas as pd

In [147]:
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
#df = pd.read_csv('WISDM_at_v2.0_raw.txt', header = None, names = columns)
#df['z-axis'] = df['z-axis'].str.rstrip(';')

In [210]:
df = pd.read_csv('users_below_id_600.csv')
df = df[df['user'] != 579]
df = df[df.columns.unique()].replace(';', '')
df = df.dropna()


for i in df['user'].unique():
    if len(df[df['user'] == i]) < 20000:
        df = df[df['user'] != i]
        print(str(i) + ': ' + str(len(df[df['user'] == i])))
 

df['timestamp'] = df['timestamp'].astype(int)
df['x-axis'] = df['x-axis'].astype(float)
#df['y-axis'] = df['y-axis'].astype(float)
#df['z-axis'] = df['z-axis'].astype(float)



599: 0
584: 0
582: 0
563: 0
568: 0
590: 0
587: 0
586: 0
573: 0
597: 0
598: 0
589: 0
593: 0
595: 0
583: 0
592: 0
591: 0
580: 0
561: 0
567: 0
581: 0
596: 0
578: 0


In [211]:
for user in df['user'].unique():
    print(user)
    for activity in df['activity'].unique():
        user_activity_to_process = df[df['user'] == user][df[df['user'] == user]['activity'] == activity]
        user_activity_to_process['timestamp'] -= user_activity_to_process['timestamp'].iloc[0]
        user_activity_to_process['timestamp'] /= 1000

594
       Unnamed: 0  user activity  timestamp  x-axis  y-axis  z-axis
1494        71100   594  Walking       0.00    3.72   10.15   -0.65
1495        71101   594  Walking       0.05    1.95    8.69    5.98
1496        71102   594  Walking       0.10    6.13    8.16   -3.53
1497        71103   594  Walking       0.15    5.86    8.73   -0.53
1498        71104   594  Walking       0.20    2.37    7.16   -0.31
...           ...   ...      ...        ...     ...     ...     ...
18314       87920   594  Walking     878.75   -1.14    4.52    0.76
18315       87921   594  Walking     878.80    2.26   10.46    5.79
18316       87922   594  Walking     878.85    2.53   13.06    9.47
18317       87923   594  Walking     878.90    1.88   10.53    6.93
18318       87924   594  Walking     878.95   -2.56   12.76    1.14

[10597 rows x 7 columns]
       Unnamed: 0  user activity  timestamp  x-axis  y-axis  z-axis
6991        76597   594  Jogging       0.00    1.53    9.28    0.65
6992        76598 

194
        Unnamed: 0  user activity    timestamp    x-axis     y-axis    z-axis
218889     1855411   194  Walking        0.000  0.823759  10.748088  0.117680
218890     1855412   194  Walking        0.050  3.334261   9.728196 -2.745862
218891     1855413   194  Walking        0.100  5.119071  13.454723 -2.628182
218892     1855414   194  Walking        0.150  4.099180  16.592852 -0.647239
218893     1855415   194  Walking        0.200  2.432049  11.964112  1.372931
...            ...   ...      ...          ...       ...        ...       ...
237276     1873798   194  Walking  1981986.497  1.361408  -0.622114 -0.045806
237277     1873799   194  Walking  1981986.547  0.762729  -2.037851 -0.268447
237278     1873800   194  Walking  1981986.597  1.301753  -0.993892 -0.627441
237279     1873801   194  Walking  1981986.647  0.857538   0.252468 -0.644485
237280     1873802   194  Walking  1981986.697 -0.217314  -0.309992 -0.483630

[12199 rows x 7 columns]
        Unnamed: 0  user activity 

        Unnamed: 0  user activity  timestamp  x-axis  y-axis    z-axis
364850     2959039   562  Walking       0.00   -3.41   18.12 -5.094010
364851     2959040   562  Walking       0.05    1.27   10.80 -0.912563
364852     2959041   562  Walking       0.10   -2.49    6.82 -2.533385
364853     2959042   562  Walking       0.15   -1.65    8.31  1.225831
364854     2959043   562  Walking       0.20    0.76   10.69 -4.290410
...            ...   ...      ...        ...     ...     ...       ...
380950     2975139   562  Walking     942.45   -2.18   11.14  0.694638
380951     2975140   562  Walking     942.50   -1.61    7.82  1.566340
380952     2975141   562  Walking     942.55   -1.50   11.73 -8.621680
380953     2975142   562  Walking     942.60   -3.79   18.43  0.994285
380954     2975143   562  Walking     942.65   -3.60   14.48  4.944186

[12201 rows x 7 columns]
        Unnamed: 0  user activity  timestamp  x-axis  y-axis    z-axis
371332     2965521   562  Jogging       0.00    8.2